In [ ]:
import pandas as pd

In [ ]:
tenis_df = pd.read_csv("dane/Dane_ATP_2000-2024 d.csv", sep=';')

In [ ]:
print(tenis_df.columns)

In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')

In [ ]:
tenis_df.head(5)

In [ ]:
#wizualizacja rozkładu powierzchni, na których grają zawodnicy
tenis_df = tenis_df.dropna(subset=['surface'])
surface = tenis_df['surface'].value_counts()
surface.plot(kind='pie', autopct='%1.1f%%', startangle=90)

In [ ]:
#zamiana kolumn ze statystykami meczowymi na ich rolling values
#lista kolumn do przetworzenia: 
stats = ['ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced']
columns_for_rolling = [f'w_{col}' for col in stats] + [f'l_{col}' for col in stats]

#dane z zawodnikami w jednej kolumnie
tenis_df['match_id'] = tenis_df.index

#przygotowanie danych dla wygranych i przegranych
winners = tenis_df[['match_id', 'tourney_date', 'winner_id'] + [f'w_{col}' for col in stats]].copy()
losers = tenis_df[['match_id', 'tourney_date', 'loser_id'] + [f'l_{col}' for col in stats]].copy()

#ujednolicenie nazw kolumn
winners.columns = ['match_id', 'tourney_date', 'player_id'] + stats
losers.columns = ['match_id', 'tourney_date', 'player_id'] + stats

#złączenie tabel winers i losers oraz posortowanie ich 
all_matches = pd.concat([winners, losers], ignore_index=True)
all_matches['tourney_date'] = pd.to_datetime(all_matches['tourney_date'], format='%Y%m%d')
all_matches = all_matches.sort_values(['player_id', 'tourney_date'])

#obliczenie średniej ruchomej dla każdego z zawodników
for col in stats:
    all_matches[f'rolling_{col}'] = all_matches.groupby('player_id')[col].transform(
        lambda x: x.rolling(window=10, min_periods=1).mean().shift(1)
    )

#mapowanie z powrotem do wygranych/przegranych
#scalanie tylko wyników, gdzie zawodnik był zwycięzcą:
winners_merged = tenis_df[['match_id', 'winner_id']].merge(
    all_matches,
    left_on=['match_id', 'winner_id'],
    right_on=['match_id', 'player_id'],
    how='left',
    suffixes=('', '_winner')
)[['match_id'] + [f'rolling_{col}' for col in stats]]

#scalanie tylko wyników, gdzie zawodnik był przegranym:
losers_merged = tenis_df[['match_id', 'loser_id']].merge(
    all_matches,
    left_on=['match_id', 'loser_id'],
    right_on=['match_id', 'player_id'],
    how='left',
    suffixes=('', '_loser')
)[['match_id'] + [f'rolling_{col}' for col in stats]]

#nadanie przejrzystości nazwom kolumn
winners_merged.columns = ['match_id'] + [f'winner_rolling_{col}' for col in stats]
losers_merged.columns = ['match_id'] + [f'loser_rolling_{col}' for col in stats]

#zmergowanie tabel z pierwotną ramką
tenis_df = tenis_df.merge(winners_merged, on='match_id', how='left')
tenis_df = tenis_df.merge(losers_merged, on='match_id', how='left')

#usuwanie zbędnych kolumn
tenis_df.drop(columns=['match_id'] + columns_for_rolling, inplace=True)

#zapisanie wyniku
tenis_df.to_csv('dane/rolling_corrected.csv', sep=';', index=False)

In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')


In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')

In [ ]:
#usunięcie kolumn, które uznaliśmy za nieistotne
columns_to_drop = [
    'tourney_id', 'tourney_name', 'draw_size', 'match_num', 'winner_seed',
    'winner_name', 'loser_seed', 'winner_ioc', 'loser_name', 'loser_seed',
    'loser_ioc', 'score', 'tourney_date', 'winner_hand', 'loser_hand'
]

tenis_df = tenis_df.drop(columns=columns_to_drop)
tenis_df.head(5)

In [ ]:
print(tenis_df.columns)

In [ ]:
#zamiana nazw kolumn: winner -> player_1 ; loser -> player_0
tenis_df.columns = tenis_df.columns.str.replace('winner', 'player1')
tenis_df.columns = tenis_df.columns.str.replace('loser', 'player0')

#sprawdzenie zmian
print(tenis_df.columns)

#dodanie kolumny label (player_1 zawsze wygrywa)
tenis_df['label'] = 1
tenis_df.head(5)


In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')

In [ ]:
print(tenis_df.dtypes)

In [ ]:
#one-hot encoding dla wybranych kolumn
columns_to_encode = ['surface', 'player0_entry', 'player1_entry']
tenis_df = pd.get_dummies(tenis_df, columns=columns_to_encode, drop_first=True)
tenis_df.head(5)


In [ ]:
#sprawdzenie, jakie wartości są w kolumnie tourney_level
print(tenis_df['tourney_level'].unique())

In [ ]:
#label encoding dla kolumny tourney_level
tourney_level_mapping = {
    'G': 6,  # Grand Slams
    'M': 5,  # Masters 1000s
    'F': 4,  # Tour finals and season-ending events
    'D': 3,  # Davis Cup
    'A': 2,  # Other tour-level events
    'C': 1,  # Challengers
    'S': 0   # Satellites/ITFs
}
tenis_df['tourney_level'] = tenis_df['tourney_level'].map(tourney_level_mapping)

In [ ]:
#sprawdzenie, jakie wartości są w kolumnie round
print(tenis_df['round'].unique())

In [ ]:
#sprawdzenie kilku losowych wierszy, w których kolumna round = ER, żeby zrozumieć jej znaczenie
er_rows = tenis_df[tenis_df['round'] == 'ER']
er_rows.tail(5)
#ER oznacza, że mecze są rozgrywane w Buenos Aires, gdzie stosuje się rundę eliminacyjną

In [ ]:
#sprawdzenie kilku losowych wierszy, w których kolumna round = RR, żeby zrozumieć jej znaczenie
rr_rows = tenis_df[tenis_df['round'] == 'RR']
rr_rows.tail(5)
#RR oznacza, że mecze są z "Davis Cup", co oznacza faze grupową (robin round)

In [ ]:
#sprawdzenie kilku losowych wierszy, w których kolumna round = BR, żeby zrozumieć jej znaczenie
br_rows = tenis_df[tenis_df['round'] == 'BR']
br_rows.tail(5)
#BR oznacza, że mecz jest z Igrzysk Olimpijskich, czyli chodzi o mecz o brązowy medal

In [ ]:
#label encoding dla kolumny round i tourney_level (BR bronze round, RR faza grupowa, ER elimination round)
round_mapping = {
    'ER': 1, 'RR': 2, 'R128': 3, 'R64': 4, 'R32': 5, 'R16': 6, 'QF': 7, 'SF': 8, 'BR': 9, 'F': 10
}
tenis_df['round'] = tenis_df['round'].map(round_mapping)

In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')

In [ ]:
#printowanie wierszy oraz kolumn, w których występują wartości NaN
null_columns = tenis_df.columns[tenis_df.isnull().any()]
print(f"Kolumny z brakującymi wartościami: {list(null_columns)}")
print("Liczba wierszy z brakującymi wartościami: ", tenis_df.isnull().sum().sum())

In [ ]:
#funkcja do uzupełniania brakujących wartości w grupach, a następnie globalnie
def fillna_grouped_then_global(df, group_col, target_col, method='mean'):
    # Uzupełnij brakujące wartości średnią/medianą w ramach grup
    if method == 'mean':
        df[target_col] = df.groupby(group_col)[target_col].transform(lambda x: x.fillna(x.mean()))
    elif method == 'median':
        df[target_col] = df.groupby(group_col)[target_col].transform(lambda x: x.fillna(x.median()))

    #uzupełnianie pozostałych brakujących wartości globalną średnią/medianą
    if method == 'mean':
        df[target_col] = df[target_col].fillna(df[target_col].mean())
    elif method == 'median':
        df[target_col] = df[target_col].fillna(df[target_col].median())

    return df

#uzupełnienie wzrostu i wieku średnią/medianą w ramach grup, a następnie globalnie
tenis_df = fillna_grouped_then_global(tenis_df, 'player1_age', 'player1_ht', method='mean')
tenis_df = fillna_grouped_then_global(tenis_df, 'player0_age', 'player0_ht', method='mean')
tenis_df['player1_age'] = tenis_df['player1_age'].fillna(tenis_df['player1_age'].median())
tenis_df['player0_age'] = tenis_df['player0_age'].fillna(tenis_df['player0_age'].median())

#uzupełnienie czasu trwania meczu medianą
tenis_df['minutes'] = tenis_df['minutes'].fillna(tenis_df['minutes'].median())

#uzupełnienie rankingu i punktów rankingowych
tenis_df['player1_rank'] = tenis_df['player1_rank'].fillna(2000)  # Poza rankingiem ATP
tenis_df['player0_rank'] = tenis_df['player0_rank'].fillna(2000)  # Poza rankingiem ATP
tenis_df['player1_rank_points'] = tenis_df['player1_rank_points'].fillna(0)  # Brak punktów
tenis_df['player0_rank_points'] = tenis_df['player0_rank_points'].fillna(0)  # Brak punktów

#dodanie flag dla brakujących wartości
tenis_df['player1_ht_missing'] = tenis_df['player1_ht'].isnull().astype(int)
tenis_df['player0_ht_missing'] = tenis_df['player0_ht'].isnull().astype(int)

player1_rolling_features = [player1_col for player1_col in tenis_df.columns if player1_col.startswith('player1_rolling_')]
player0_rolling_features = [player0_col for player0_col in tenis_df.columns if player0_col.startswith('player0_rolling_')]
rolling_features = player1_rolling_features + player0_rolling_features

#uzupełnienie statystyk rollingowych wartością domyślną (0)
for col in rolling_features:
    tenis_df[col] = tenis_df[col].fillna(0)

#sprawdzenie, czy wszystkie brakujące wartości zostały wyeliminowane
print("Czy nadal istnieją brakujące wartości w danych?", tenis_df.isnull().sum().sum() > 0)

#pozostale kolumny z brakującymi wartościami
null_columns = tenis_df.columns[tenis_df.isnull().any()]
print(f"Kolumny z brakującymi wartościami: {list(null_columns)}")

In [ ]:
#sprawdzenie ilości wierszy (powinno byc 73247)
print(tenis_df.shape[0])
#sprawdzenie ile razy id (103819) Rogera Federera wystepuje jako wygrany (powinno byc 1250)
print(tenis_df[tenis_df['player1_id'] == 103819].shape[0])

In [ ]:
#nie możemy w kolumnie Y mieć tylko jednej wartości, dlatego dla połowy przypadków zamieniamy zawodnika 1. z zawodnikiem 2. i label z 1 na 0
def swap_players_every_other_row(df):
    #dodanie kolumny player1_entry_S, jeśli jej nie ma
    if 'player1_entry_S' not in df.columns:
        df['player1_entry_S'] = 0

    #filtrowanie kolumn dla player1 i player0
    player1_cols = [column for column in df.columns if column.startswith('player1')]
    player0_cols = [column for column in df.columns if column.startswith('player0')]

    #upewnienie się, że kolumny player1 i player0 mają taką samą strukturę
    assert len(player1_cols) == len(player0_cols), "kolumny player1 i player0 muszą być zgodne."

    #wybranie indeksów parzystych
    swap_indices = df.index[::2]

    #zamiana danych player1 <-> player0 w wybranych wierszach
    #upewnienie się, że typy danych są zgodne
    for col1, col0 in zip(player1_cols, player0_cols):
        df[col1] = df[col1].astype(df[col0].dtype)
        df[col0] = df[col0].astype(df[col1].dtype)

    df.loc[swap_indices, player1_cols + player0_cols] = df.loc[swap_indices, player0_cols + player1_cols].values

    #ustawienie wartości label na 0 w podmienionych wierszach
    if 'label' in df.columns:
        df.loc[swap_indices, 'label'] = 0

    return df

swapped_players_df = swap_players_every_other_row(tenis_df)
print(swapped_players_df.shape[0], 'liczba rowow, powinno byc 73247')
swapped_players_df.head(5)

In [ ]:
#liczymy ile wystąpiło upsetów (różnica między rankingami - 10)
#"upset" - sytuacja, w której zawodnik z gorszym rankingiem wygrał mecz

#ustawienie progu różnicy rankingowej, czyli sytuacji, w której różnica rankingów jest na tyle duża, że można mówić o upsetach
ranking_threshold = 10

#sprawdzenie, czy wystąpił upset z uwzględnieniem progu różnicy rankingowej
tenis_df['is_upset'] = (
    ((tenis_df['label'] == 1) & (tenis_df['player1_rank'] - tenis_df['player0_rank'] > ranking_threshold)) |  # Player1 wygrał, ale miał znacznie gorszy ranking
    ((tenis_df['label'] == 0) & (tenis_df['player0_rank'] - tenis_df['player1_rank'] > ranking_threshold))    # Player0 wygrał, ale miał znacznie gorszy ranking
)

#liczba upsetów
num_upsets = tenis_df['is_upset'].sum()

#obliczenie procent upsetów
percent_upsets = (num_upsets / len(tenis_df)) * 100

print(f"Procent upsetów (z progiem różnicy rankingowej {ranking_threshold}): {percent_upsets:.2f}%")

In [ ]:
#liczymy ile wystąpiło upsetów (różnica między rankingami - 100)

#ustawienie progu różnicy rankingowej, czyli sytuacji, w której różnica rankingów jest na tyle duża, że można mówić o upsetach
ranking_threshold = 100

#sprawdzenie, czy wystąpił upset z uwzględnieniem progu różnicy rankingowej
tenis_df['is_upset'] = (
    ((tenis_df['label'] == 1) & (tenis_df['player1_rank'] - tenis_df['player0_rank'] > ranking_threshold)) |  # Player1 wygrał, ale miał znacznie gorszy ranking
    ((tenis_df['label'] == 0) & (tenis_df['player0_rank'] - tenis_df['player1_rank'] > ranking_threshold))    # Player0 wygrał, ale miał znacznie gorszy ranking
)

#liczba upsetów
num_upsets = tenis_df['is_upset'].sum()

#obliczenie procent upsetów
percent_upsets = (num_upsets / len(tenis_df)) * 100

print(f"Procent upsetów (z progiem różnicy rankingowej {ranking_threshold}): {percent_upsets:.2f}%")

In [ ]:
#usunięcie kolumny is_upset
tenis_df = tenis_df.drop(columns=['is_upset'])

In [ ]:
print(tenis_df.shape[0], 'liczba rowow, powinno byc 73247')

In [ ]:
swapped_players_df.to_csv('dane/gotowe_dane.csv', sep=';', index=False)